In [ ]:
using CausalityTools
include("../entropy.jl")
using Random
using BenchmarkTools

rng = MersenneTwister(145)

a = rand(rng, 0:10, 100)
b = rand(rng, 0:10, 100)

est = Kraskov(k=1)

print("Current method :")
foo = @btime TE(Int.(a .> 0), Int.(b .> 0))
println("The result is $foo.\n")


print("True transfer entropy :")
a = float.(a)
b = float.(b)
foo2 = @btime transferentropy(a, b, est)
println("The result is $foo2.\n")

print("CCM :")
foo3 = @btime crossmap(a, b, 2, 1)
println("The result is $foo3.\n")

In [ ]:
import PyPlot as plt

rng = MersenneTwister(145)

a = rand(rng, 0:10, 100)
b = rand(rng, 0:10, 100)

a = float.(a)
b = float.(b)

Ls = [10:5:50; 60:10:100]

@btime begin
    test = [crossmap(a[1:L], b[1:L], 2, 1) for L in Ls]
    test2 = [crossmap(b[1:L], a[1:L], 2, 1) for L in Ls]
end

plt.figure()
plt.plot(Ls, test, "b-", label="a to b")
plt.plot(Ls, test2, "r-", label="b to a")
plt.legend()
show(plt.gcf())

In [ ]:
using CausalityTools

s_measure()

In [14]:
using CausalityTools

# A two-dimensional Ulam lattice map
sys = ulam(2)

# Sample 1000 points after discarding 5000 transients
orbit = trajectory(sys, 1000, Ttr = 5000)
x, y = orbit[:, 1], orbit[:, 2]

# 4-dimensional embedding for `x`, 5-dimensional embedding for `y`
s_measure(x, y, dx = 4, τx = 3, dy = 5, τy = 1)

MethodError: MethodError: no method matching s_measure(::Vector{Float64}, ::Vector{Float64}; dx=4, τx=3, my=5, τy=1)
Closest candidates are:
  s_measure(::AbstractVector{T}, ::AbstractVector{T}; K, dx, dy, τx, τy, metric, tree_metric) where T at ~/.julia/packages/CausalityTools/Zxxnz/src/SMeasure/smeasure.jl:140 got unsupported keyword argument "my"
  s_measure(!Matched::DelayEmbeddings.AbstractDataset{D}, ::AbstractVector{T}; K, dy, τy, metric, tree_metric) where {D, T} at ~/.julia/packages/CausalityTools/Zxxnz/src/SMeasure/smeasure.jl:155 got unsupported keyword arguments "dx", "τx", "my"
  s_measure(::AbstractVector{T}, !Matched::DelayEmbeddings.AbstractDataset{D}; K, dx, τx, metric, tree_metric) where {D, T} at ~/.julia/packages/CausalityTools/Zxxnz/src/SMeasure/smeasure.jl:163 got unsupported keyword arguments "my", "τy"

In [23]:
using DelayEmbeddings

a = collect(1:100)
p = embed(a, 2, 5)

2-dimensional Dataset{Int64} with 95 points
  1    6
  2    7
  3    8
  4    9
  5   10
  6   11
  7   12
  8   13
  9   14
 10   15
  ⋮  
 87   92
 88   93
 89   94
 90   95
 91   96
 92   97
 93   98
 94   99
 95  100

In [26]:
using Neighborhood

a = [0, 1.2, 0.1, 0.8]
b = [0, 0, 1.8, 0.8]


X = embed(a, 2, 1)
Y = embed(b, 2, 1)

treeX = searchstructure(KDTree, X, Euclidean())
treeY = searchstructure(KDTree, Y, Euclidean())
neighborhoodtype, theiler = NeighborNumber(1), Theiler(0)
idxs_X = bulkisearch(treeX, X, neighborhoodtype, theiler)
idxs_Y = bulkisearch(treeY, Y, neighborhoodtype, theiler)

print(idxs_Y)

LoadError: UndefVarError: embed not defined

In [15]:
Vector{Int64} <: AbstractArray

true

In [30]:
using NearestNeighbors

get_T(::Type{T}) where {T <: AbstractFloat} = T
get_T(::T) where {T} = Float64


get_T (generic function with 2 methods)

In [34]:
a = rand(1:10, 10)

foo = typemax(get_T(Int))

fill!(a, foo)

LoadError: InexactError: Int64(Inf)

In [38]:
using BenchmarkTools

@btime float(a)

  38.222 ns (1 allocation: 144 bytes)


10-element Vector{Float64}:
  3.0
 10.0
  1.0
  1.0
  9.0
  3.0
  4.0
  3.0
  5.0
  5.0

In [27]:
using CausalityTools
a = collect(1:10)
b = rand(1:10, 10)

# s_measure(float.(a), float.(b))
s_measure(a, b)

LoadError: InexactError: Int64(Inf)

In [22]:
using Neighborhood

a = rand(1:10, 10, 5)
typeof(a)

treeX = searchstructure(KDTree, a, Euclidean())

LoadError: MethodError: no method matching KDTree(::Matrix{Int64}, ::Euclidean)
[0mClosest candidates are:
[0m  KDTree([91m::AbstractVecOrMat{T}[39m, ::M; leafsize, storedata, reorder, reorderbuffer) where {T<:AbstractFloat, M<:Union{Chebyshev, Cityblock, Euclidean, Minkowski, WeightedCityblock, WeightedEuclidean, WeightedMinkowski}} at ~/.julia/packages/NearestNeighbors/8gDgr/src/kd_tree.jl:85
[0m  KDTree([91m::AbstractVector{V}[39m, ::M; leafsize, storedata, reorder, reorderbuffer) where {V<:AbstractArray, M<:Union{Chebyshev, Cityblock, Euclidean, Minkowski, WeightedCityblock, WeightedEuclidean, WeightedMinkowski}} at ~/.julia/packages/NearestNeighbors/8gDgr/src/kd_tree.jl:27
[0m  KDTree([91m::DelayEmbeddings.AbstractDataset[39m, ::Distances.Metric; kwargs...) at ~/.julia/packages/DelayEmbeddings/yBNbA/src/datasets/neighborhoods.jl:9

In [20]:
Matrix{Int64} <: AbstractVecOrMat

true

In [23]:
using NearestNeighbors

KDTree(a)

LoadError: MethodError: no method matching KDTree(::Matrix{Int64})
[0mClosest candidates are:
[0m  KDTree([91m::Vector{V}[39m, [91m::NearestNeighbors.HyperRectangle{T}[39m, [91m::Vector{Int64}[39m, [91m::M[39m, [91m::Array{NearestNeighbors.KDNode{T}, 1}[39m, [91m::NearestNeighbors.TreeData[39m, [91m::Bool[39m) where {V<:(AbstractVector), M<:Union{Chebyshev, Cityblock, Euclidean, Minkowski, WeightedCityblock, WeightedEuclidean, WeightedMinkowski}, T} at ~/.julia/packages/NearestNeighbors/8gDgr/src/kd_tree.jl:11
[0m  KDTree([91m::AbstractVecOrMat{T}[39m) where T<:AbstractFloat at ~/.julia/packages/NearestNeighbors/8gDgr/src/kd_tree.jl:85
[0m  KDTree([91m::AbstractVecOrMat{T}[39m, [91m::M[39m; leafsize, storedata, reorder, reorderbuffer) where {T<:AbstractFloat, M<:Union{Chebyshev, Cityblock, Euclidean, Minkowski, WeightedCityblock, WeightedEuclidean, WeightedMinkowski}} at ~/.julia/packages/NearestNeighbors/8gDgr/src/kd_tree.jl:85
[0m  ...

In [24]:
Int64 <: AbstractFloat

false

In [25]:
eltype(a)

Int64